In [1]:
# 本章で共通して利用するライブラリの読み込み。
import os
import pandas as pd
from glob import glob

In [2]:
# コード 6.1 収益率と超過収益率の計算方法

indexCSV='./in/index.csv'
closeCSV='./in/close.csv'
# インデックスの終値の読み込みと収益率の計算
# parse_dates=で日付型として認識させたい列名を指定する。
idf=pd.read_csv(indexCSV,parse_dates=['date'])
# 個別銘柄の列名も'close'のため、インデックスの方の名称を'iclose'に変更する。
idf=idf.rename(columns={'close':'iclose'})
# この一行で収益率の計算ができている。
idf['ireturn']=idf['iclose'].pct_change()

# 銘柄の終値の読み込みと収益率の計算
cdf=pd.read_csv(closeCSV,parse_dates=['date'],dtype={'ticker':'object'})
# インデックスのときと違い、銘柄別に収益率を計算する必要があるため、groupbyを使っている。
cdf['return']=cdf[['ticker','close']].groupby(['ticker']).pct_change()

# 銘柄の収益率にインデックスの終値を結合し、超過収益率を計算
df=pd.merge(cdf,idf,on='date') # 結合
df['areturn']=df['return']-df['ireturn'] # 超過収益率の計算
# 先頭行(初日)は収益率が計算できないのでNAになっており、その行を削除している。
df=df.dropna()
df=df.sort_values(['ticker','date'])
df

,ticker,date,close,return,iclose,ireturn,areturn
2,0001,2007-04-18,120,0.200000,450,0.125000,0.075000
4,0001,2007-04-19,120,0.000000,460,0.022222,-0.022222
6,0001,2007-04-20,130,0.083333,430,-0.065217,0.148551
3,0002,2007-04-18,5210,0.001923,450,0.125000,-0.123077
5,0002,2007-04-19,5180,-0.005758,460,0.022222,-0.027980
7,0002,2007-04-20,5150,-0.005792,430,-0.065217,0.059426


In [3]:
# コード 6.2 行をシフトすることによる収益率の計算 (shift を使う方法)
# コード 6.2と違うところのみコメントを入れている。

indexCSV='./in/index.csv'
closeCSV='./in/close.csv'
idf=pd.read_csv(indexCSV,parse_dates=['date'])
idf=idf.rename(columns={'close':'iclose'})

# pct_change()の代わりに以下の２行で、より一般的な方法で収益率が求まる。
idf['iclose2']=idf['iclose'].shift()
idf['ireturn']=idf['iclose']/idf['iclose2']-1
########

cdf=pd.read_csv(closeCSV,parse_dates=['date'],dtype={'ticker':'object'})

# 銘柄別の方も同様に以下の２行で収益率が求まる。
cdf['close2']=cdf[['ticker','close']].groupby(['ticker']).shift()
cdf['return']=cdf['close']/cdf['close2']-1
########

df=pd.merge(cdf,idf,on='date') # 結合
df['areturn']=df['return']-df['ireturn'] # 超過収益率の計算
df=df.dropna()
df=df.sort_values(['ticker','date'])
df

,ticker,date,close,close2,return,iclose,iclose2,ireturn,areturn
2,0001,2007-04-18,120,100.0,0.200000,450,400.0,0.125000,0.075000
4,0001,2007-04-19,120,120.0,0.000000,460,450.0,0.022222,-0.022222
6,0001,2007-04-20,130,120.0,0.083333,430,460.0,-0.065217,0.148551
3,0002,2007-04-18,5210,5200.0,0.001923,450,400.0,0.125000,-0.123077
5,0002,2007-04-19,5180,5210.0,-0.005758,460,450.0,0.022222,-0.027980
7,0002,2007-04-20,5150,5180.0,-0.005792,430,460.0,-0.065217,0.059426


In [4]:
# コード 6.3 リストと辞書を使った分配金調整
# 関数として実装している。
def adjustDivi(priceCSV,diviCSV):
    # 日付順の累積計算やNaNの埋め込みが控えているので、読み込み後に日付で並べ替えておく。
    price=pd.read_csv(priceCSV,parse_dates=['date']).sort_values('date')
    div=pd.read_csv(diviCSV,parse_dates=['date']).sort_values('date')
    div['div_cum']=div['dividend'].cumsum() # 分配金を累積したもの
    # diviCSVの日付は、priceCSVの日付の一部なので、
    # 結合するときはleftのouter joinすることで、日付で結合できなくてもpriceの方は全行出力される。
    pTable=pd.merge(price,div,on='date',how='left')

    # outer joinで分配のない日の分配金はNaNになっている。
    # NaNを直近(csv読み込み時にdate順に並んでる)の分配金で埋める。
    # 先頭行からNaNがある場合は、fillna(method='ffill')では直近がないためNaNのままとなるので、
    # それをfillna(0)により0で埋める。
    pTable['div_fill']=pTable['div_cum'].fillna(method='ffill').fillna(0)
    # 価額と累積分配金を足し込むことで価額の調整終了
    pTable['price_adj']=pTable['price']+pTable['div_fill']
    
    # 日付、オリジナルの価額、調整後の価額のDataFrameを返す。
    return pTable[['date','price','div_fill','price_adj']]

priceCSV='./in/price.csv'
diviCSV ='./in/dividend.csv'
price_adj=adjustDivi(priceCSV,diviCSV)
price_adj

,date,price,div_fill,price_adj
0,2007-04-17,100,0.0,100.0
1,2007-04-18,120,2.0,122.0
2,2007-04-19,110,2.0,112.0
3,2007-04-20,130,7.0,137.0
4,2007-04-23,120,7.0,127.0
5,2007-04-24,140,8.0,148.0
6,2007-04-25,138,8.0,146.0


In [5]:
# コード 2.1、コード 2.2の再掲(インターネットに接続できなければ実行できない)

# 以下のURLでにアクセスすれば、３要因データの様々なバリエーション、
# weeklyやmonthly、そして他の国用のデータも揃っている。
# https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
import urllib
import zipfile

url='http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/Japan_3_Factors_Daily_CSV.zip'
urllib.request.urlretrieve(url,'./out/3factor.zip')

with zipfile.ZipFile('./out/3factor.zip') as zf:
    # zipファイルの中にアーカイブされているフィアル一覧を表示する
    print(zf.namelist())
    # zipファイルから取り出すファイル名と、取り出し先にパス名を指定する。
    zf.extract('Japan_3_Factors_Daily.csv', './')


['Japan_3_Factors_Daily.csv']


In [6]:
# コード 6.4 3要因モデルデータのクリーニング処理

# 先頭６行はデータの説明が入っているのでスキップする(skiprows=6)。
# 先頭列は日付なので、parse_dates=[0]で、0番目の項目を日付型に変換することになる。
# 同じく日付を行のインデックスにする(index_col=[0])。
fact3=pd.read_csv('./Japan_3_Factors_Daily.csv', skiprows=6, parse_dates=[0],index_col=[0])
fact3.index.name='date' # 0番目の項目は名称がないので、'date'と名付ける。

# 先頭行とデータ型を確認する。
print(fact3.head())
print(fact3.dtypes)

# CSVに出力
# 次回からは、pd.read_csv('./3factors.csv')のみで読み込み可能になる。
fact3.to_csv('./out/3factors.csv')


            Mkt-RF   SMB   HML    RF
date                                
1990-07-02    0.85  0.38 -0.06  0.03
1990-07-03    0.07  0.72  0.30  0.03
1990-07-04    1.45  0.52  0.26  0.03
1990-07-05   -0.64  0.48  0.38  0.03
1990-07-06   -0.02  0.40  0.31  0.03
Mkt-RF    float64
SMB       float64
HML       float64
RF        float64
dtype: object


In [7]:
# コード 6.5 ファイル名リストを作成するスクリプト
# ファイルリストの作成

def getTickers(iPath):
    tickers={}
    # globでワイルドカードにマッチする全ての「分配金」ファイル名を取得する。
    # *は任意の文字列にマッチすることを意味する。
    # for文にて、ファイル名を一つづつdivFile変数にセットして、インデントブロックの処理を回していく。
    for diviFile in glob(iPath+'*/*/*_divi.csv'):
        # pathの区切り文字である'/'でファイル名を分割し、全要素をnames変数にリストで格納する。
        # windowsとunix系OSで区切り文字が異なるため、共通のコードを書くためにos.sepを用いている。
        names=diviFile.split(os.sep) # ['.', 'in', '7', 'dividend', 'large', 'value', '0007_divi.csv']
        # tickerシンボル、value/gloth、small/largeの各属性値を変数にセットする。
        # リストのマイナスの数字は、後ろから数えての要素を取得している。
        ticker=names[-1].replace('_divi.csv','')
        gv    =names[-2]
        size  =names[-3]
        # 価額ファイル名は、分配金ファイル名の末尾から'_divi'を削除したものなので、ここでセットしておく。
        priceFile=diviFile.replace('_divi','')
        # tickerシンボルをキーにして属性を辞書にセットする。
        tickers[ticker]=[priceFile,diviFile,size,gv]
    # for文を抜けて、全ての銘柄がセットされたtickers辞書を返す。
    return tickers

fileList=getTickers('./in/dividend/')
fileList


{'0007': ['./in/dividend/large/value/0007.csv',
  './in/dividend/large/value/0007_divi.csv',
  'large',
  'value'],
 '0008': ['./in/dividend/large/value/0008.csv',
  './in/dividend/large/value/0008_divi.csv',
  'large',
  'value'],
 '0006': ['./in/dividend/large/growth/0006.csv',
  './in/dividend/large/growth/0006_divi.csv',
  'large',
  'growth'],
 '0005': ['./in/dividend/large/growth/0005.csv',
  './in/dividend/large/growth/0005_divi.csv',
  'large',
  'growth'],
 '0003': ['./in/dividend/small/value/0003.csv',
  './in/dividend/small/value/0003_divi.csv',
  'small',
  'value'],
 '0004': ['./in/dividend/small/value/0004.csv',
  './in/dividend/small/value/0004_divi.csv',
  'small',
  'value'],
 '0001': ['./in/dividend/small/growth/0001.csv',
  './in/dividend/small/growth/0001_divi.csv',
  'small',
  'growth'],
 '0002': ['./in/dividend/small/growth/0002.csv',
  './in/dividend/small/growth/0002_divi.csv',
  'small',
  'growth']}

In [8]:
# コード 6.6 全ファイルの価額調整をし、3要因モデルを結合したデータセットを作成

# 作成されたデータセットを保存するフォルダの作成。
os.makedirs('./out/adjust/',exist_ok=True)

# 直前のセルにて作成したファイル名一覧fileListで回す。
# keyは銘柄コード、valは価額ファイル名や分配金ファイル名などその属性リスト。
for key,val in fileList.items():
    # わかりやすさのため、リストの要素を変数にセットする。
    priceCSV=val[0]
    diviCSV=val[1]
    size=val[2]
    gvType=val[3]
    # 前に作成した分配金調整の関数を呼び出す。
    adjust=adjustDivi(priceCSV,diviCSV)
    # 収益率の計算
    adjust['return']=adjust['price_adj'].pct_change()
    # ３要因データの結合
    adjust=adjust.merge(fact3,on='date')
    # 各種属性値のセット
    adjust['ticker']=key
    adjust['size']=size
    adjust['gvType']=gvType
    # 日付をインデックスにセットしてNA行を削除
    adjust=adjust.set_index('date')
    adjust=adjust.dropna()
    # 列の並びを整理する。
    adjust=adjust[['ticker','size','gvType','price','div_fill','price_adj','return','Mkt-RF','SMB','HML']]
    # 銘柄名.csvにてCSVファイルに出力
    adjust.to_csv('./out/adjust/'+key+'.csv')
    # 確認のため画面出力する。
    display(adjust.head())
    

,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
date,,,,,,,,,,
1996-04-01,0007,large,value,9675,0.0,9675.0,-0.032500,0.48,0.21,0.12
1996-04-02,0007,large,value,9675,0.0,9675.0,0.000000,0.00,0.61,0.21
1996-04-03,0007,large,value,9753,0.0,9753.0,0.008062,0.45,0.55,-0.04
1996-04-04,0007,large,value,9661,0.0,9661.0,-0.009433,-0.33,0.61,-0.18
1996-04-05,0007,large,value,10000,0.0,10000.0,0.035090,0.42,-0.05,-0.13


,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
date,,,,,,,,,,
1991-06-04,0008,large,value,9757,0.0,9757.0,-0.024300,-0.51,0.48,0.30
1991-06-05,0008,large,value,9925,0.0,9925.0,0.017218,-1.07,0.41,0.10
1991-06-06,0008,large,value,10000,0.0,10000.0,0.007557,-0.62,-0.22,0.31
1991-06-07,0008,large,value,10278,0.0,10278.0,0.027800,-0.78,0.27,0.06
1991-06-10,0008,large,value,10454,0.0,10454.0,0.017124,-2.42,-0.09,0.50


,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
date,,,,,,,,,,
2001-11-21,0006,large,growth,10000,0.0,10000.0,0.000000,-0.32,-0.34,0.85
2001-11-22,0006,large,growth,10055,0.0,10055.0,0.005500,-0.23,-0.04,-0.24
2001-11-26,0006,large,growth,10214,0.0,10214.0,0.015813,2.54,-0.76,-0.79
2001-11-27,0006,large,growth,9962,0.0,9962.0,-0.024672,-0.67,0.59,-0.13
2001-11-28,0006,large,growth,10715,0.0,10715.0,0.075587,-1.65,0.73,0.54


,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
date,,,,,,,,,,
1998-10-22,0005,large,growth,11049,0.0,11049.0,0.104900,-0.35,-0.02,1.46
1998-10-23,0005,large,growth,10627,0.0,10627.0,-0.038194,-0.96,0.50,-0.67
1998-10-26,0005,large,growth,10310,0.0,10310.0,-0.029830,-2.75,1.44,0.00
1998-10-27,0005,large,growth,10296,0.0,10296.0,-0.001358,0.54,0.33,-1.12
1998-10-28,0005,large,growth,10240,0.0,10240.0,-0.005439,-1.25,0.74,0.19


,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
date,,,,,,,,,,
2009-07-03,0003,small,value,9958,0.0,9958.0,-0.004200,-0.41,0.37,0.11
2009-07-06,0003,small,value,9969,0.0,9969.0,0.001105,-0.08,0.76,-1.07
2009-07-07,0003,small,value,10053,0.0,10053.0,0.008426,0.24,0.27,-0.99
2009-07-08,0003,small,value,10064,0.0,10064.0,0.001094,-0.08,0.85,-1.51
2009-07-09,0003,small,value,10022,0.0,10022.0,-0.004173,-1.68,0.63,-0.93


,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
date,,,,,,,,,,
2001-10-31,0004,small,value,9927,0.0,9927.0,-0.007300,-1.05,0.54,0.79
2001-11-01,0004,small,value,10590,0.0,10590.0,0.066788,0.17,-0.39,-0.97
2001-11-02,0004,small,value,10163,0.0,10163.0,-0.040321,-0.09,0.11,-0.55
2001-11-05,0004,small,value,9971,0.0,9971.0,-0.018892,0.08,-0.06,0.77
2001-11-06,0004,small,value,9912,0.0,9912.0,-0.005917,1.22,-0.35,-1.20


,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
date,,,,,,,,,,
2009-07-03,0001,small,growth,9933,0.0,9933.0,-0.006700,-0.41,0.37,0.11
2009-07-06,0001,small,growth,9888,0.0,9888.0,-0.004530,-0.08,0.76,-1.07
2009-07-07,0001,small,growth,9944,0.0,9944.0,0.005663,0.24,0.27,-0.99
2009-07-08,0001,small,growth,10373,0.0,10373.0,0.043142,-0.08,0.85,-1.51
2009-07-09,0001,small,growth,10258,0.0,10258.0,-0.011086,-1.68,0.63,-0.93


,ticker,size,gvType,price,div_fill,price_adj,return,Mkt-RF,SMB,HML
date,,,,,,,,,,
2008-07-03,0002,small,growth,9801,0.0,9801.0,-0.019900,-1.12,-0.41,-0.26
2008-07-04,0002,small,growth,9366,0.0,9366.0,-0.044383,-0.06,0.05,0.26
2008-07-07,0002,small,growth,9584,0.0,9584.0,0.023276,0.68,-0.30,-0.07
2008-07-08,0002,small,growth,9783,0.0,9783.0,0.020764,-2.35,0.00,0.28
2008-07-09,0002,small,growth,9783,0.0,9783.0,0.000000,0.71,-0.13,0.04


In [9]:
# 章末問題 1) 銘柄別に収益率と超過収益率の平均と標準偏差を求めなさい

indexCSV='./in/index.csv'
closeCSV='./in/close.csv'
idf=pd.read_csv(indexCSV,parse_dates=['date'])
idf=idf.rename(columns={'close':'iclose'})
idf['ireturn']=idf['iclose'].pct_change()
cdf=pd.read_csv(closeCSV,parse_dates=['date'],dtype={'ticker':'object'})
cdf['return']=cdf[['ticker','close']].groupby(['ticker']).pct_change()
df=pd.merge(cdf,idf,on='date') # 結合
df['areturn']=df['return']-df['ireturn'] # 超過収益率の計算
df=df.dropna()
df=df.sort_values(['ticker','date'])
# ここまでがpct_changeを用いた収益率の計算結果

# 銘柄別収益率平均
display(df[['ticker','return','areturn']].groupby(['ticker']).mean())
# 銘柄別収益率標準偏差
display(df[['ticker','return','areturn']].groupby(['ticker']).std())

# describe()を用いれば７つの基本統計量を一気に出力できる。
# ただし、複数の変数(returnとareturn)を対象とすると、列はMutiindexの形式で出力され、
# 列(columns)は変数と７つの基本統計量の木構造となる。
display(df[['ticker','return','areturn']].groupby(['ticker']).describe())


,return,areturn
ticker,,
0001,0.094444,0.067110
0002,-0.003209,-0.030544


,return,areturn
ticker,,
0001,0.100462,0.085659
0002,0.004444,0.091278


return                                                              \
        count      mean       std       min       25%       50%       75%   
ticker                                                                      
0001      3.0  0.094444  0.100462  0.000000  0.041667  0.083333  0.141667   
0002      3.0 -0.003209  0.004444 -0.005792 -0.005775 -0.005758 -0.001918   

                 areturn                                                   \
             max   count      mean       std       min       25%      50%   
ticker                                                                      
0001    0.200000     3.0  0.067110  0.085659 -0.022222  0.026389  0.07500   
0002    0.001923     3.0 -0.030544  0.091278 -0.123077 -0.075529 -0.02798   

                            
             75%       max  
ticker                      
0001    0.111775  0.148551  
0002    0.015723  0.059426

In [10]:
# 章末問題 2) 収益率の定義を「収益率=close_t/close_{t-1}」として計算する。


closeCSV='./in/close.csv'
df=pd.read_csv(closeCSV,parse_dates=['date'],dtype={'ticker':'object'})
df['close2']=df[['ticker','close']].groupby(['ticker']).shift()
# オリジナルの収益率の計算から-1が省かれている。
df['return']=df['close']/df['close2']

close3=[] # ここに収益率から計算されるcloseの値を格納していく
# iterrowsで、行のindexと値のタップルでループする
for index,row in df.iterrows():
    # 銘柄の先頭行は収益率の計算ができないのでNaNになっており、それを判定することで先頭行を判別する。
    if pd.isnull(row['return']):
        # 先頭行は元のcloseの値を出力
        close3.append(row['close'])
    else:
        # それ以外の行は直前の結果(close3[-1])に収益率を掛けて元のcloseの値が復元される。
        close3.append(row['return']*close3[-1])
# 元のDataFrameに結果をセットして完了
df['close3']=close3
df


,ticker,date,close,close2,return,close3
0,0001,2007-04-17,100,NaN,NaN,100.0
1,0001,2007-04-18,120,100.0,1.200000,120.0
2,0001,2007-04-19,120,120.0,1.000000,120.0
3,0001,2007-04-20,130,120.0,1.083333,130.0
4,0002,2007-04-17,5200,NaN,NaN,5200.0
5,0002,2007-04-18,5210,5200.0,1.001923,5210.0
6,0002,2007-04-19,5180,5210.0,0.994242,5180.0
7,0002,2007-04-20,5150,5180.0,0.994208,5150.0


In [11]:
# 章末問題 3) pandasのpct_chageやshiftといった機能を用いずに、for文で収益率を計算する方法。

indexCSV='./in/index.csv'
closeCSV='./in/close.csv'
idf=pd.read_csv(indexCSV,parse_dates=['date'])
idf=idf.rename(columns={'close':'iclose'})

# for文で、一行ずつcloseを読み込んで前行のcloseと計算して収益率を求める。
# 日付順に並べる。CSVが日付順に並んでいればよいが、その保証はないのでここで並べておく。
idf=idf.sort_values('date')
c=idf['iclose'] # close列は以下で頻繁に使うので変数cにセットしておく
r=[None] # このリストに収益率を行ごとに追加していく。先頭行は計算できないのでNoneをセットしておく。
# 行で回す。0行目でなく1行目から回すのがポイント。
for i in range(1,len(c)):
    r.append(c[i]/c[i-1]-1) # i行目とその前のi-1行目で収益率の計算
idf['ireturn']=r # DataFrameにセット

cdf=pd.read_csv(closeCSV,parse_dates=['date'],dtype={'ticker':'object'})

# 銘柄別の方もインデックス(idf)と同様だが、ticker別に処理しなければならないのでひと手間増える。
cdf=cdf.sort_values(['ticker','date'])
t=cdf['ticker']
c=cdf['close']
r=[]
for i in range(len(c)):
    if i==0 or t[i]!=t[i-1]: # ticker別の先頭行の時にマッチする条件
        r.append(None)
    else:
        r.append(c[i]/c[i-1]-1) # i行目とその前のi-1行目で収益率の計算
cdf['return']=r # DataFrameにセット
########

df=pd.merge(cdf,idf,on='date') # 結合
df['areturn']=df['return']-df['ireturn'] # 超過収益率の計算
df=df.dropna()
df=df.sort_values(['ticker','date'])
df

,ticker,date,close,return,iclose,ireturn,areturn
2,0001,2007-04-18,120,0.200000,450,0.125000,0.075000
4,0001,2007-04-19,120,0.000000,460,0.022222,-0.022222
6,0001,2007-04-20,130,0.083333,430,-0.065217,0.148551
3,0002,2007-04-18,5210,0.001923,450,0.125000,-0.123077
5,0002,2007-04-19,5180,-0.005758,460,0.022222,-0.027980
7,0002,2007-04-20,5150,-0.005792,430,-0.065217,0.059426


In [12]:
# 章末問題 4) 分配金調整のコード7.3について、cumsum()とmerge()を用いず、分配金データを辞書に格納し、
# for文で一行づつ価額調整を行うプログラムを作成しなさい。
# 関数形式で実装し、出力形式はコード7.3と同様とする。
def adjustDivi3(priceCSV,diviCSV):
    # 日付順の累積計算やNAの埋め込みが控えているので、読み込み後に日付で並べ替えておく。
    price=pd.read_csv(priceCSV,parse_dates=['date']).sort_values('date')
    div=pd.read_csv(diviCSV,parse_dates=['date']).sort_values('date')
    # print(div)
    #         date  dividend
    # 1 2007-04-18         2
    # 2 2007-04-20         5
    # 0 2007-04-24         1

    # この辞書変数に日付をキーにした分配金を格納していく
    date_div={}
    for index,row in div.iterrows():
        date_div[row['date']]=row['dividend']
    # print(date_div)
    # {Timestamp('2007-04-18 00:00:00'): 2, Timestamp('2007-04-20 00:00:00'): 5, Timestamp('2007-04-24 00:00:00'): 1}

    div_fill=[] # 分配金の累積を格納していく
    price_adj=[] # 調整後価額を格納していく
    accum=0.0 # 累積値の現在値
    for index,row in price.iterrows():
        # 価額データのdateがdate_divに存在するということは、その日は分配の日であるということ。
        if row['date'] in date_div: # 分配の日であれば
            accum+=date_div[row['date']] # accum変数に分配金を累積していく
        div_fill.append(accum) # その値をリストに格納
        price_adj.append(row['price']+accum) # 調整価額の計算
    # 元のDataFrameにdiv_fillとprice_adjをセットする
    price['div_fill']=div_fill
    price['price_adj']=price_adj
    
    # 新たなDataFrameを返して収量
    return price

priceCSV='./in/price.csv'
diviCSV ='./in/dividend.csv'
price_adj=adjustDivi3(priceCSV,diviCSV)
display(price_adj)


,date,price,div_fill,price_adj
0,2007-04-17,100,0.0,100.0
1,2007-04-18,120,2.0,122.0
2,2007-04-19,110,2.0,112.0
3,2007-04-20,130,7.0,137.0
4,2007-04-23,120,7.0,127.0
5,2007-04-24,140,8.0,148.0
6,2007-04-25,138,8.0,146.0
